# 대화 생성 모델

### 문제 상황
진료 시간 변경 요청1 : “당신의 이름은 홍길동 이고, 전화번호는 010-1234-5678입니다. 오늘인 2025.07.01 화요일에 경희 내과에 예약이 되어 있습니다. 그런데 갑자기 일정이 생겨 시간이 어렵게 되었습니다. 병원에 전화해 진료 시간을 오후로 변경 가능한지 요청해보세요.”

# 1. environment setting


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 라이브러리 설치
!pip install transformers datasets
# 앞에 !를 붙이는 이유는 python 코드가 아니라, 터미널 명령어를 직접 실행하겠다는 뜻

!pip install --upgrade transformers


# 2. model training


기본 모델, training data 불러오기


In [5]:
# 1. 라이브러리 임포트
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch
import sys
import pandas as pd

# 2. 학습에 앞서 GPU를 사용하고 있는지 check
if not torch.cuda.is_available():
    print("현재 GPU가 설정되어 있지 않습니다. Colab 메뉴에서 런타임 유형 변경 > 하드웨어 가속기 : GPU로 바꿔주세요.")
    sys.exit()


# 3. dataset 불러오기 (CSV)
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Finetuning_medical_fixed/dialogues_medical_fixed(utf-8).csv")  # 드라이브에서 해당 문서 위치 보고 경로 수정

# 4. HuggingFace Dataset으로 변환
dataset = Dataset.from_pandas(df)

print("\nCSV 파일에서 상위 5개 행 출력\n")
print(df.head())  # 기본적으로 상위 5개 행 출력
print("\nHuggingFace Dataset에서 샘플 출력\n")
print(dataset[:5])  # 처음 5개 샘플 출력
print(df.columns) # column명 출력

# 5. train_dataset & test_dataset으로 나누기
from datasets import DatasetDict

split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


CSV 파일에서 상위 5개 행 출력

                                              prompt  \
0  안녕하세요. 오늘 경희내과에 예약이 되어 있는데요, 시간이 좀 어려워서 변경 문의드...   
1                           홍길동입니다. 오늘 오전에 예약돼 있었어요.   
2              오후 시간대로 바꾸고 싶은데요, 가능한 시간 알려주실 수 있을까요?   
3                                   3시 10분으로 부탁드릴게요.   
4                                             감사합니다.   

                                          completion  
0                 네, 안녕하세요. 확인 도와드릴게요. 성함이 어떻게 되실까요?  
1  잠시만요.확인되었습니다. 오늘 오전 10시 30분, 내과 진료로 예약되어 있으시네요...  
2  네, 오후에는 2시 20분, 3시 10분, 그리고 4시가 비어 있습니다. 어떤 시간...  
3        네, 3시 10분으로 변경해드렸습니다. 진료 10분 전까지 도착 부탁드립니다.  
4                            네, 조심히 오세요. 좋은 하루 보내세요.  

HuggingFace Dataset에서 샘플 출력

{'prompt': ['안녕하세요. 오늘 경희내과에 예약이 되어 있는데요, 시간이 좀 어려워서 변경 문의드리려고요.', '홍길동입니다. 오늘 오전에 예약돼 있었어요.', '오후 시간대로 바꾸고 싶은데요, 가능한 시간 알려주실 수 있을까요?', '3시 10분으로 부탁드릴게요.', '감사합니다.'], 'completion': ['네, 안녕하세요. 확인 도와드릴게요. 성함이 어떻게 되실까요?', '잠시만요.확인되었습니다. 오늘 오전 10시 30분, 내과 진료로 예약되어 있으시네요. 변경 원하시는 시간대 

training 설정


In [6]:
# 5. 모델과 토크나이저 불러오기
model_name = "hyunwoongko/kobart"       # gogamza/kobart-base-v2
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 학습 전에 모델을 GPU에 올리기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 6. 전처리 함수 정의
def preprocess_function(example):
    model_inputs = tokenizer(example["prompt"], max_length=64, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["completion"], max_length=64, truncation=True, padding="max_length")

    # padding된 label은 -100으로 마스킹
    labels["input_ids"] = [ (l if l != tokenizer.pad_token_id else -100) for l in labels["input_ids"] ]
    # labels에 pad 토큰 마스킹이 필요한 이유
    # completion은 길이가 다 달라서 padding으로 짧은 문장은 공백으로 채우게 되어 있음. 이게 pad_token
    # 이 pad도 label로 쓰면 모델이 그걸 예측하려고 하는 문제 발생.

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# train, eval 모두 전처리
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)
# KoBART는 BART 기반이라 token_type_ids가 필요 없다 캄. 그래서 token_type_ids도 제외시켜줘야 됨.
tokenized_train = tokenized_train.remove_columns(["prompt", "completion"])   # , "token_type_ids"
tokenized_eval = tokenized_eval.remove_columns(["prompt", "completion"])

# 7. 학습 설정
training_args = TrainingArguments(
    output_dir="./kobart-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=8,
    learning_rate=5e-5,
    logging_steps=10,
    remove_unused_columns=False,  # trainer은 기본적으로 모델에 필요한 컬럼 이외의 것을 제거하려고 함.
    # 그래서 이렇게 하면 prompt, completion 컬럼이 남아있어도 무시하지 않고 그냥 넘어감.
)

# 8. Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

# 9. 파인튜닝 시작
trainer.train()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


Map:   0%|          | 0/789 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/198 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wjd_bin217 (wjd_bin217-kyung-hee-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,10.232500
20,4.335300
30,2.843100
40,1.506300
50,0.933200
60,0.514600
70,0.409800
80,0.642800
90,0.510100
100,0.376900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3160, training_loss=0.180935456214732, metrics={'train_runtime': 435.351, 'train_samples_per_second': 14.499, 'train_steps_per_second': 7.259, 'total_flos': 240541032775680.0, 'train_loss': 0.180935456214732, 'epoch': 8.0})

# 3. evaluation

In [7]:
eval_results = trainer.evaluate()
print(f"\n평가 결과:\n{eval_results}")


평가 결과:
{'eval_loss': 0.3384854197502136, 'eval_runtime': 1.0495, 'eval_samples_per_second': 188.67, 'eval_steps_per_second': 23.822, 'epoch': 8.0}


# 4. inference

standard version

In [8]:
def generate_response(prompt_text, max_length=64):
    model.eval()  # 평가 모드
    input_ids = tokenizer(prompt_text, return_tensors="pt", truncation=True, padding="max_length", max_length=64).input_ids.to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            num_beams=4,              # 빔 서치로 더 나은 응답 유도
            early_stopping=True,
            no_repeat_ngram_size=2,   # 같은 n-gram 반복 방지
            repetition_penalty=1.5,   # 반복 억제
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [9]:
# 테스트 입력 예시
test_prompts = [
    "안녕하세요. 오늘 경희내과에 예약이 되어 있는데요, 시간이 좀 어려워서 변경 문의드리려고요.",
    "홍길동입니다. 오늘 오전에 예약돼 있었어요.",
    "오후 시간대로 바꾸고 싶은데요, 가능한 시간 알려주실 수 있을까요?",
    "3시 10분으로 부탁드릴게요.",
    "네 감사합니다.",
]

# 결과 출력
for i, prompt in enumerate(test_prompts):
    print(f"[입력 {i+1}] {prompt}")
    print(f"[응답 {i+1}] {generate_response(prompt)}\n")


[입력 1] 안녕하세요. 오늘 경희내과에 예약이 되어 있는데요, 시간이 좀 어려워서 변경 문의드리려고요.
[응답 1] 네, 안녕하세요. 확인 도와드릴게요. 성함이 어떻게 되실까요?

[입력 2] 홍길동입니다. 오늘 오전에 예약돼 있었어요.
[응답 2] 잠시만요.확인되었습니다. 오늘 오전 11시 내과 진료로 예약되어 있으시네요. 오후에 가능한 시간은 2시 30분, 3시 50분, 5시 10분입니다.

[입력 3] 오후 시간대로 바꾸고 싶은데요, 가능한 시간 알려주실 수 있을까요?
[응답 3] 네, 오후 2시 30분, 3시 50분, 5시 10분 진료가 가능합니다.

[입력 4] 3시 10분으로 부탁드릴게요.
[응답 4] 네, 3시 10분으로 변경해드리겠습니다. 진료 10분 전까지 도착 부탁드립니다.

[입력 5] 네 감사합니다.
[응답 5] 네, 조심히 오세요.



angry version


In [12]:
# 테스트할 문장 목록
test_sentences = [
    "아, 진짜. 개빡치네. 경희내과 맞아요? 진료 시간 좀 바꾸려고하는데. 진짜 짜증나네.",
    "나 홍길동인데, 오늘 오전 11시에 예약됐다고. 근데 갑자기 못 가게 됐다고! 오후에 언제 비는데",
    "뭔 시간이 이따위야? 3시 50분밖에 없어?",
    "하.... 알았어. 3시 50분으로 비워놔.",
    "끊어.",
]

# 모델을 평가 모드로 전환
model.eval()

print("\n=== 테스트 문장 응답 ===")
for sentence in test_sentences:
    # 토크나이징
    encoded = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=64)
    # token_type_ids 제거 (KoBART에서 필요 없음)
    if 'token_type_ids' in encoded:
        del encoded['token_type_ids']

    # GPU 사용 가능하면 모델/입력 데이터를 GPU로 이동
    encoded = {k: v.to(device) for k, v in encoded.items()}

    # 응답 생성
    output_ids = model.generate(
        **encoded,
        max_length=64,
        num_beams=5,
        length_penalty=1.0,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # 디코딩 후 출력
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"사용자: {sentence}")
    print(f"모델 응답: {response}\n")



=== 테스트 문장 응답 ===
사용자: 아, 진짜. 개빡치네. 경희내과 맞아요? 진료 시간 좀 바꾸려고하는데. 진짜 짜증나네.
모델 응답: 네, 불편하시겠지만 어떤 문의이실까요?

사용자: 나 홍길동인데, 오늘 오전 11시에 예약됐다고. 근데 갑자기 못 가게 됐다고! 오후에 언제 비는데
모델 응답: 예약 내역은 확인됩니다. 오늘 오후에는 2시 30분, 3시 50분, 5시 10분이 비어 있습니다.

사용자: 뭔 시간이 이따위야? 3시 50분밖에 없어?
모델 응답: 현재 그 시간들이 가능합니다.

사용자: 하.... 알았어. 3시 50분으로 비워놔.
모델 응답: 네, 그렇게 변경해드리겠습니다.

사용자: 끊어.
모델 응답: 네.



# 5. model save

In [13]:
save_path = "/content/drive/MyDrive/Colab Notebooks/Finetuning_medical_fixed/medical_fintuned(kobart-hyunwoongko)"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/Colab Notebooks/Finetuning_medical_fixed/medical_fintuned(kobart-hyunwoongko)/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Finetuning_medical_fixed/medical_fintuned(kobart-hyunwoongko)/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Finetuning_medical_fixed/medical_fintuned(kobart-hyunwoongko)/tokenizer.json')

# 6. uploading to Hugging Face

In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
model.push_to_hub("medical_fintuned_kobart-hyunwoongko")      # hugging face hub에 생성될 모델 저장소의 이름
tokenizer.push_to_hub("medical_fintuned_kobart-hyunwoongko")

Uploading...:   0%|          | 0.00/496M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wjdbin217/medical_fintuned_kobart-hyunwoongko/commit/15d3e88f34678e12931d761d15dcffbe94dc5464', commit_message='Upload tokenizer', commit_description='', oid='15d3e88f34678e12931d761d15dcffbe94dc5464', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wjdbin217/medical_fintuned_kobart-hyunwoongko', endpoint='https://huggingface.co', repo_type='model', repo_id='wjdbin217/medical_fintuned_kobart-hyunwoongko'), pr_revision=None, pr_num=None)